In [3]:
import numpy as np
import time

from sklearn import datasets
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, FastICA

from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot

#Always run this the command before at the start of notebook
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# Number of dimensions
ndims = 3
scatterClass = go.Scatter3d
if ndims == 2:
    scatterClass = go.Scatter
    
# Label of the plot
plot_title = 'Visualise.html'

# Dimensionality reduction
reduce_funcs = {
    'ica': FastICA,
    'pca': PCA
}

In [4]:
def get_mnist_dataset():
    digits = datasets.load_digits()
    data = digits.data
    labels = digits.target
    return data, labels


def convert_mnist_labels_to_string(labels):
    return np.array([str(label) for label in labels])


def perform_dim_reduction(kind, data, n_components):
    dim_reduce_kind = reduce_funcs[kind]
    func = dim_reduce_kind(n_components=n_components)
    data_reduced = func.fit_transform(data, y=None)
    if hasattr(func, 'explained_variance_ratio_'):
        print('Cumulative explained variation for {} principal components: {}'
              .format(n_components, np.sum(dim_reduce_func.explained_variance_ratio_)))


def perform_tsne(data):    
    tsne = TSNE(n_components=ndims, verbose=1, perplexity=10, n_iter=4000)
    time_start = time.time()
    result = tsne.fit_transform(data)
    print('t-SNE done! Time elapsed: {} seconds'.format(time.time() - time_start))
    return result


def perform_mds(data, n_iter=1000):
    from sklearn.manifold import MDS
    model = MDS(n_components=ndims, dissimilarity='euclidean', random_state=7, verbose=1, max_iter=n_iter)
    result = model.fit_transform(data)
    stress = model.stress_
    print('Stress value is: {}'.format(stress))
    return result


In [12]:
data, labels = get_mnist_dataset()
labels = convert_mnist_labels_to_string(labels)
unique_labels = np.unique(labels)
data_tsne = perform_mds(data)

ValueError: array must be 2-dimensional and square. shape = (1797, 64)

In [4]:
traces = []
for label in unique_labels:
    ind = np.where(labels==label)
    coords = dict(
        x=data_tsne[ind, 0].ravel(),
        y=data_tsne[ind, 1].ravel()
    )
    if ndims == 3:
        coords['z'] = data_tsne[ind, 2].ravel()

    ids = labels[ind];

    trace = scatterClass(
        name=label.strip(),
        mode='markers',
        marker=dict(
            size=5,
            line=dict(
                width=0.5
            ),
            opacity=1
        ),
        text=ids,
        **coords
    )
    traces.append(trace)
layout = go.Layout(
    hovermode='closest',
    title=plot_title,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)


fig = go.Figure(data=traces, layout=layout)
iplot(fig, filename=plot_title, show_link = True)

In [5]:
import plotly.plotly as py
import plotly.graph_objs as go

py.iplot(fig, filename=plot_title, show_link = True)